# PASO 2: SUPONGAMOS QUE NOS INGRESA: "LOTE, PRODUCTO, CANTIDAD"

In [ ]:
# MODELO

nec = {
    ('lot1', 'Y4W'): 220,
    ('lot2', 'Q79'): 123,
    ('lot3', 'VEQ'): 134,
}

prod = {
    'Y4W': 
    {
    'corte_m': 0,
    'doblado': 0,
    'soldado': 0,
    'corte_w': 23,
    'tapizado': 0,
    'pintura': 15,
    'despacho': 24
    },
    'Q79': 
    {
    'corte_m': 45,
    'doblado': 23,
    'soldado': 32,
    'corte_w': 34,
    'tapizado': 102,
    'pintura': 36,
    'despacho': 34
    },
    'VEQ': 
    {
    'corte_m': 48,
    'doblado': 38,
    'soldado': 56,
    'corte_w': 0,
    'tapizado': 0,
    'pintura': 36,
    'despacho': 20
    }
}

op = ['corte_m','doblado','soldado','corte_w','tapizado','pintura','despacho']

ultima_op = 'despacho'

from docplex.cp.model import CpoModel

m = CpoModel()

ope = []

# intervalos
for i in nec:
    r = []
    for j in op:      
        r.append(m.interval_var(optional = True, size = prod[i[1]][j]*nec[i], name = i[0] + '_' + i[1] + '_' +  j ))
    ope.append(r)

# relaciones de precedencia

for i in nec:
    idx = list(nec.keys()).index(i) # consulta de stackoverflow

    if i[1] == 'Y4W':   # estanteria de madera
        m.add(m.presence_of(ope[idx][op.index('corte_m')])==0)
        m.add(m.presence_of(ope[idx][op.index('doblado')])==0)
        m.add(m.presence_of(ope[idx][op.index('soldado')])==0)
        m.add(m.presence_of(ope[idx][op.index('corte_w')])==1)
        m.add(m.presence_of(ope[idx][op.index('tapizado')])==0)
        m.add(m.presence_of(ope[idx][op.index('pintura')])==1)
        m.add(m.presence_of(ope[idx][op.index('despacho')])==1)
        m.add(m.end_before_start(ope[idx][op.index('corte_w')], ope[idx][op.index('pintura')]))
        m.add(m.end_before_start(ope[idx][op.index('pintura')], ope[idx][op.index('despacho')], 24*60))
    if i[1] == 'Q79':   # silla combinada
        
        m.add(m.presence_of(ope[idx][op.index('corte_m')])==1)
        m.add(m.presence_of(ope[idx][op.index('doblado')])==1)
        m.add(m.presence_of(ope[idx][op.index('soldado')])==1)
        m.add(m.presence_of(ope[idx][op.index('corte_w')])==1)
        m.add(m.presence_of(ope[idx][op.index('tapizado')])==1)
        m.add(m.presence_of(ope[idx][op.index('pintura')])==1)
        m.add(m.presence_of(ope[idx][op.index('despacho')])==1) 
        m.add(m.end_before_start(ope[idx][op.index('corte_m')], ope[idx][op.index('doblado')]))
        m.add(m.end_before_start(ope[idx][op.index('doblado')], ope[idx][op.index('soldado')]))        
        m.add(m.end_at_start(ope[idx][op.index('soldado')], ope[idx][op.index('pintura')]))  
        m.add(m.end_before_start(ope[idx][op.index('pintura')], ope[idx][op.index('despacho')], 24*60))
        m.add(m.end_before_start(ope[idx][op.index('corte_w')], ope[idx][op.index('tapizado')]))
        m.add(m.end_before_start(ope[idx][op.index('tapizado')], ope[idx][op.index('despacho')]))
    if i[1] == 'VEQ':   # silla de metal
        
        m.add(m.presence_of(ope[idx][op.index('corte_m')])==1)
        m.add(m.presence_of(ope[idx][op.index('doblado')])==1)
        m.add(m.presence_of(ope[idx][op.index('soldado')])==1)
        m.add(m.presence_of(ope[idx][op.index('corte_w')])==0)
        m.add(m.presence_of(ope[idx][op.index('tapizado')])==0)
        m.add(m.presence_of(ope[idx][op.index('pintura')])==1)
        m.add(m.presence_of(ope[idx][op.index('despacho')])==1)
        m.add(m.end_before_start(ope[idx][op.index('corte_m')], ope[idx][op.index('doblado')]))
        m.add(m.end_before_start(ope[idx][op.index('doblado')], ope[idx][op.index('soldado')]))        
        m.add(m.end_at_start(ope[idx][op.index('soldado')], ope[idx][op.index('pintura')]))  
        m.add(m.end_before_start(ope[idx][op.index('pintura')], ope[idx][op.index('despacho')], 24*60))

# funcional
m.add( m.minimize(m.max(m.end_of(ope[i][op.index(ultima_op)]) for i in range(len(nec)))))

# resolución
s = m.solve()

s.print_solution()



 ! --------------------------------------------------- CP Optimizer 22.1.0.0 --
 ! Minimization problem - 21 variables, 33 constraints
 ! Initial process time : 0.02s (0.02s extraction + 0.00s propagation)
 !  . Log search space  : 58.6 (before), 58.6 (after)
 !  . Memory usage      : 304.1 kB (before), 304.1 kB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0         21                 -
 + New bound is 27972
 ! Using iterative diving.
 *         27972       10  0.04s        1      (gap is 0.00%)
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! Best objective         : 27972 (optimal - effective tol. is 2)
 ! Best bound             : 27972
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 

In [2]:
print(m.export_as_cpo())

///////////////////////////////////////////////////////////////////////////////
// CPO file generated at 2025.04.01-22:11:07 for model: <ipython-input-1-cfffceac5d64>
// Source file: <ipython-input-1-cfffceac5d64>
///////////////////////////////////////////////////////////////////////////////

//--- Constants ---

//--- Variables ---
#line 65 "<ipython-input-1-cfffceac5d64>"
lot1_Y4W_corte_m = intervalVar(optional, size=0);
#line 66
lot1_Y4W_doblado = intervalVar(optional, size=0);
#line 67
lot1_Y4W_soldado = intervalVar(optional, size=0);
#line 68
lot1_Y4W_corte_w = intervalVar(optional, size=5060);
#line 69
lot1_Y4W_tapizado = intervalVar(optional, size=0);
#line 70
lot1_Y4W_pintura = intervalVar(optional, size=3300);
#line 71
lot1_Y4W_despacho = intervalVar(optional, size=5280);
#line 77
lot2_Q79_corte_m = intervalVar(optional, size=5535);
#line 78
lot2_Q79_doblado = intervalVar(optional, size=2829);
#line 79
lot2_Q79_soldado = intervalVar(optional, size=3936);
#line 80
lot2_Q79_cor